In [2]:
import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sympy
import datetime

# для расчетов над tdb
from pycalphad import Database, binplot

# для MCMC расчетов
import pymc as pm  # пакет для MCMC расчетов 
import arviz as az # пакет для работы с типом данных arviz
import pytensor
import pytensor.tensor as pt
# import theano
# theano.config.exception_verbosity = 'high' # должно выдавать подробное описание ошибки, но не помогает

import aesara

import seaborn as sns

# пути к tdb
db18_path = 'tdbs/CoCr-18Cac_with_new_functions.tdb'
db10_path = 'tdbs/CoCr-01Oik_with_new_functions.tdb'

print(f"Running on PyMC v{pm.__version__}") # 5.3.1 or 5.1.2
print(f"Running on NumPy v{np.__version__}") # 1.22.1
print(f"Running on ArviZ v{az.__version__}") # 0.12.1

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC v5.3.1
Running on NumPy v1.22.1
Running on ArviZ v0.15.1


In [3]:
# для 0.5
df_sigma_fcc = pd.read_excel('emp_data/sigma_fcc_allibert.xls')
df_sigma_hcp = pd.read_excel('emp_data/sigma_hcp_allibert.xls')

df_hcp_fcc = pd.concat([df_sigma_fcc, df_sigma_hcp])

df_hcp_fcc['T'] = df_hcp_fcc['T'].round(2)
df_hcp_fcc['cr_conc'] = df_hcp_fcc['cr_conc'].round(6)
df_hcp_fcc = df_hcp_fcc[(df_hcp_fcc['phase'] == 'sigma_old')].reset_index()
df_hcp_fcc.sort_values('T', inplace=True)
df_hcp_fcc

,index,cr_conc,T,phase
6,7,0.541762,928.09,sigma_old
9,10,0.558231,973.03,sigma_old
7,8,0.545313,1020.22,sigma_old
8,9,0.551960,1069.66,sigma_old
10,11,0.559991,1121.35,sigma_old
12,13,0.565255,1168.54,sigma_old
11,12,0.560713,1217.98,sigma_old
1,7,0.542469,1226.97,sigma_old
4,10,0.556156,1269.66,sigma_old
0,6,0.532019,1321.35,sigma_old


In [4]:
# для 0.75
df_bcc = pd.read_excel('emp_data/sigma_bcc_allibert.xls')

df_bcc['T'] = df_bcc['T'].round(2)
df_bcc['cr_conc'] = df_bcc['cr_conc'].round(6)
df_bcc = df_bcc[(df_bcc['phase'] == 'sigma_old')].reset_index()
df_bcc.sort_values('T', inplace=True)
# df_bcc

In [5]:
trace18 = az.from_json('calc_res/trace_cocr18_2Sx700x1000x2_20230509.json')
ppc18 = az.from_json('calc_res/ppc_cocr18_2Sx700x1000x2_20230509.json')
pp18 = az.from_json('calc_res/pp_cocr18_2Sx2000_20230509.json')
main18 = trace18.extend(ppc18)
par_list18 = ['GSCRCO1', 'GSCOCRCO1', 'GSCOCRCO2', 'GSCRCO2', 'GSCOCR1',  'GSCOCR2', 'GSCOCR3']

trace10 = az.from_json('calc_res/trace_cocr10_2Sx700x1000x4.json')
ppc10 = az.from_json('calc_res/ppc_cocr10_2Sx700x1000x4.json')
pp10 = az.from_json('calc_res/pp_cocr10_2Sx4000.json')
main10 = trace10.extend(ppc10)
par_list10 = ['SIGMA_OLD_COCRCO_0', 'SIGMA_OLD_COCRCO_1', 'SIGMA_OLD_COCRCR_0', 'SIGMA_OLD_COCRCR_1']

# BF

$$BF = \frac{P(Y|M_1)}{P(Y|M_2)} = exp(\frac{P(Y|M_1)}{P(Y|M_2)}) = exp(mean(P(Y|M_1)) - mean(P(Y|M_2)))$$

In [13]:
BF = np.exp(trace10.log_likelihood.mean() - trace18.log_likelihood.mean())
np.round(BF)

c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\xarray\core\computation.py:760: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)


<xarray.Dataset>
Dimensions:    ()
Data variables:
    y_norm_05  float64 4.428e+140
    y_norm_75  float64 inf

# WAIC

In [8]:
az.waic(trace10, var_name='y_norm_05')

c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 4000 posterior samples and 13 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -1202.62   399.97
p_waic      167.09        -

There has been a warning during the calculation. Please check the results.

In [9]:
az.waic(trace10, var_name='y_norm_75')

c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 4000 posterior samples and 8 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -3523.84   963.74
p_waic      342.16        -

There has been a warning during the calculation. Please check the results.

In [10]:
az.waic(trace18, var_name='y_norm_05')

Computed from 2000 posterior samples and 13 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -5314.55  1101.24
p_waic        0.07        -

In [11]:
az.waic(trace18, var_name='y_norm_05')


Computed from 2000 posterior samples and 13 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -5314.55  1101.24
p_waic        0.07        -

# Отношение апостериорных вероятностей

Апостериорная вероятность 

$$P(M|Y) = \frac{P(Y|M)P(M)}{P(Y)} = \frac{P(Y|M)P(M)}{\int{P(Y|M)P(M)}{dM}} = ?$$